In [31]:
# importing necessary libraries
import pandas as pd
import numpy as np

# reading cab rides data into dataframe
rides_df = pd.read_csv('dataOriginal/cab_rides.csv')

# reading weather data into dataframe
weather_df = pd.read_csv('dataOriginal/weather.csv')



In [40]:
# 
rides_df['time_stamp'] = pd.to_datetime(rides_df['time_stamp'], unit='ms').dt.floor('h')

weather_df['time_stamp'] = pd.to_datetime(rides_df['time_stamp'], unit='ms').dt.floor('h')


In [47]:
# removing Null Values from rides dataframe 

rides_df



In [48]:
# Remove product id from rides df


In [ ]:
# Converting id from 30 characters -> 0, 1, 2, etc..


In [41]:
# splitting dataframes into Uber and Lyft
uber_df = rides_df[rides_df['cab_type'] == 'Uber']
lyft_df = rides_df[rides_df['cab_type'] == 'Lyft']